In [1]:
import os
import numpy as np
import tensorflow as tf
from model import *
from tensorflow.python import debug as tf_debug

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# Misc
LOG_DIR = './log/'
LABELS ='./labels_1024.tsv'
SPRITES = './sprite_1024.png'

# Training Parameters
learning_rate = 0.001
num_steps = 500
batch_size = 128
display_step = 10

# Network Parameters
num_classes = 10
keep_rate = 0.75

In [4]:
model_graph = tf.Graph()
with model_graph.as_default():
    
    images = tf.placeholder(tf.float32, [None, 28, 28, 1])
    labels = tf.placeholder(tf.float32, [None, num_classes])
    keep_prob = tf.placeholder(tf.float32)
    
    tf.summary.image('input', images, 3)
    
    # Logits and Predictions
    logits, embedding_input = build_model(images, labels, keep_prob) 
    prediction = {'classes': tf.argmax(logits, axis=1), 
                  'prob': tf.nn.softmax(logits, name='prob')}
    
    # Loss and optimizer
    with tf.variable_scope('cross_entropy_loss'):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
        tf.summary.scalar('cross_entropy_loss', loss)
    
    with tf.variable_scope('train'):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    
    # Evaluation
    with tf.variable_scope('accuracy'):
        correct_pred = tf.equal(prediction['classes'], tf.argmax(labels, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        tf.summary.scalar('accuracy', accuracy)
    
    # Initializer
    initializer = tf.global_variables_initializer()
    
    # Summary
    summary = tf.summary.merge_all()
    
    # Embedding
    embedding = tf.Variable(tf.zeros([1024, 1024]), name='test_embedding')
    assignment = embedding.assign(embedding_input)
    saver = tf.train.Saver()
    
    config = tf.contrib.tensorboard.plugins.projector.ProjectorConfig()
    embedding_config = config.embeddings.add()
    embedding_config.tensor_name = embedding.name
    
#     embedding_config.sprite.image_path = SPRITES
    embedding_config.metadata_path = 'labels.tsv'
#     # Specify the width and height of a single thumbnail.
#     embedding_config.sprite.single_image_dim.extend([28, 28])
    

In [6]:
with tf.Session(graph=model_graph) as sess:
    
    #sess = tf_debug.TensorBoardDebugWrapperSession(sess, "MZQ-MBP.local:7000")
    
    writer = tf.summary.FileWriter('./log/model_1', graph=model_graph)
    
    tf.contrib.tensorboard.plugins.projector.visualize_embeddings(writer, config)
    
    # Run Initializer
    sess.run(initializer)
    
    # Training
    for step in range(num_steps):
        
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape([-1, 28, 28, 1])
        
        sess.run(train_op, feed_dict={images: batch_x, labels: batch_y, keep_prob: keep_rate})
        
        if step % display_step == 0:
            
            loss_val, acc, s = sess.run([loss, accuracy, summary],
                                        feed_dict={images: batch_x, labels: batch_y, keep_prob: 1.0})
            
            writer.add_summary(s, step)
            
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss_val) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
            
        if step % 100 == 0:
            
            sess.run(assignment, feed_dict={images: mnist.test.images[:1024].reshape([-1, 28, 28, 1]),
                                            labels: mnist.test.labels[:1024]})
            saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), step)
            
    writer.close()

Step 0, Minibatch Loss= 2.4499, Training Accuracy= 0.141
Step 10, Minibatch Loss= 1.6004, Training Accuracy= 0.625
Step 20, Minibatch Loss= 0.6396, Training Accuracy= 0.812
Step 30, Minibatch Loss= 0.5619, Training Accuracy= 0.828
Step 40, Minibatch Loss= 0.3923, Training Accuracy= 0.922
Step 50, Minibatch Loss= 0.2996, Training Accuracy= 0.922
Step 60, Minibatch Loss= 0.0923, Training Accuracy= 0.977
Step 70, Minibatch Loss= 0.2619, Training Accuracy= 0.906
Step 80, Minibatch Loss= 0.1456, Training Accuracy= 0.953
Step 90, Minibatch Loss= 0.2114, Training Accuracy= 0.930
Step 100, Minibatch Loss= 0.1130, Training Accuracy= 0.984
Step 110, Minibatch Loss= 0.2531, Training Accuracy= 0.906
Step 120, Minibatch Loss= 0.0730, Training Accuracy= 0.977
Step 130, Minibatch Loss= 0.0829, Training Accuracy= 0.984
Step 140, Minibatch Loss= 0.0490, Training Accuracy= 0.992
Step 150, Minibatch Loss= 0.1006, Training Accuracy= 0.969
Step 160, Minibatch Loss= 0.0847, Training Accuracy= 0.977
Step 170

In [11]:
mnist.train.num_examples

55000

In [10]:
writer = tf.summary.FileWriter('./model_1/', graph=model_graph)